In [28]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
pd.set_option("mode.chained_assignment", None)

import warnings
warnings.filterwarnings("ignore")

import tkinter as tk
from tkinter import filedialog

from datetime import datetime
import dateutil.relativedelta

## Transform stock status

In [ ]:
class InventoryCheck:
   
   def __init__(self) -> None:
       self.path = self.get_file_path()
       self.df = self.open_file()
       self.result = self.data_reshaping()
       
   
   def get_file_path(self):
      root = tk.Tk()
      root.withdraw()
      path = filedialog.askopenfilename()
      return path
   
   def open_file(self):
      df = pd.read_excel(self.path)
      return df      


   def data_reshaping(self):
      ref_data = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\ReferenceTable\20221123_PCODE整理表.xlsx")
      dimension = dict(zip(ref_data["PCODE"], ref_data["規格"]))
      pattern = dict(zip(ref_data["PCODE"], ref_data["花紋"]))
      origin = dict(zip(ref_data["PCODE"], ref_data["產地"]))
      tire_type = dict(zip(ref_data["PCODE"], ref_data["Group"]))
      self.df["規格"] = self.df["物料號碼"].map(dimension)
      self.df["花紋"] = self.df["物料號碼"].map(pattern)
      self.df["產地"] = self.df["物料號碼"].map(origin)
      self.df["胎別"] = self.df["物料號碼"].map(tire_type)
      self.df["產地"] = self.df["產地"].map(lambda x: "TH" if x in ["BTMT", "TBSC"] else "JP" if x == "BSJ" else "CN")
      self.df.columns = self.df.columns.str.strip()
      self.df = self.df[['胎別', '規格', '花紋', '產地', '吋別', '新工', '景山', '彰濱', '貨櫃',
         '新工REP', '景山REP', '彰濱REP', '貨櫃REP', '本社', '景山', '林口', '凍結', 'A倉', '台中', '嘉義', '台南', '高雄']]
      self.df = self.df.groupby(["胎別", "規格", "花紋", "產地", "吋別"]).sum().stack().reset_index()
      self.df.columns = ['胎別', '規格', '花紋', '產地', '吋別', '倉庫別', "數量"]
      self.df = self.df.astype({"數量": int})
      export_path = self.get_file_path()
      self.df.to_excel(export_path, index=False)
      return self.df

In [ ]:
result = InventoryCheck().result
